In [11]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [12]:
cig_data = pd.read_csv("/Users/ellenwu/homework3_attempt1-/data/input/The_Tax_Burden_on_Tobacco__1970-2019_20250304.csv")
cpi_data = pd.read_excel("/Users/ellenwu/homework3_attempt1-/data/input/SeriesReport-20250304165118_4f02a8.xlsx", skiprows=11)

In [3]:
cig_data["Year"] = cig_data["Year"].astype(int)
cpi_data["Year"] = cpi_data["Year"].astype(int)

In [5]:
# Cleaning Tobacco Data
cig_data["measure"] = cig_data["SubMeasureDesc"].map({
    "Average Cost per pack": "cost_per_pack",
    "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
    "Federal and State tax as a Percentage of Retail Price": "tax_percent",
    "Federal and State Tax per pack": "tax_dollar",
    "Gross Cigarette Tax Revenue": "tax_revenue",
    "State Tax per pack": "tax_state"
})
cig_data = cig_data.rename(columns={"LocationAbbr": "state_abb", "LocationDesc": "state", "Data_Value": "value"})
cig_data = cig_data[["state_abb", "state", "Year", "value", "measure"]]

In [6]:
# Pivot Data
final_data = cig_data.pivot(index=["state", "Year"], columns="measure", values="value").reset_index()

# 🛠️ Fix 2: Drop incorrect year values before merging
final_data = final_data[(final_data["Year"] >= 1970) & (final_data["Year"] <= 2019)]

# Cleaning CPI Data
print("Columns in DataFrame after skipping rows:", cpi_data.columns.tolist()) 

Columns in DataFrame after skipping rows: ['Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'HALF1', 'HALF2']


In [7]:
# 🛠️ Fix 3: Drop 'HALF1' and 'HALF2' columns from CPI dataset
cpi_data = cpi_data.drop(columns=["HALF1", "HALF2"], errors="ignore")

cpi_data = cpi_data.melt(id_vars=["Year"], 
                          value_vars=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                          var_name="month", 
                          value_name="index")

cpi_data = cpi_data.groupby("Year", as_index=False).agg({"index": "mean"})

In [8]:
# Merge CPI Data
final_data = final_data.merge(cpi_data, on="Year", how="left")  # 🛠️ Fix 4: Use 'left' instead of 'outer'

# 🛠️ Fix 5: Fill Missing CPI Values
final_data["index"].fillna(final_data["index"].mean(), inplace=True)

# Adjust to 2010 Dollars
final_data["price_cpi"] = final_data["cost_per_pack"] * (218 / final_data["index"])

In [9]:
# Check Fixed Year Range
print(final_data[["Year"]].drop_duplicates().sort_values("Year"))

    Year
0   1970
1   1971
2   1972
3   1973
4   1974
5   1975
6   1976
7   1977
8   1978
9   1979
10  1980
11  1981
12  1982
13  1983
14  1984
15  1985
16  1986
17  1987
18  1988
19  1989
20  1990
21  1991
22  1992
23  1993
24  1994
25  1995
26  1996
27  1997
28  1998
29  1999
30  2000
31  2001
32  2002
33  2003
34  2004
35  2005
36  2006
37  2007
38  2008
39  2009
40  2010
41  2011
42  2012
43  2013
44  2014
45  2015
46  2016
47  2017
48  2018
49  2019


In [10]:
# Save Data
final_data.to_csv("/Users/ellenwu/homework3/data/output/ftaxburden_data.csv", index=False)

In [13]:
df = pd.read_csv("/Users/ellenwu/homework3/data/output/ftaxburden_data.csv")
print("Unique Years in Original Data:", df["Year"].unique())  

Unique Years in Original Data: [1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983
 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997
 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011
 2012 2013 2014 2015 2016 2017 2018 2019]
